M5 Forecasting - Accuracy

Note: This is one of the two complementary competitions that together comprise the M5 forecasting challenge. Can you estimate, as precisely as possible, the point forecasts of the unit sales of various products sold in the USA by Walmart?

How much camping gear will one store sell each month in a year? To the uninitiated, calculating sales at this level may seem as difficult as predicting the weather. Both types of forecasting rely on science and historical data. While a wrong weather forecast may result in you carrying around an umbrella on a sunny day, inaccurate business forecasts could result in actual or opportunity losses. In this competition, in addition to traditional forecasting methods you’re also challenged to use machine learning to improve forecast accuracy.

The Makridakis Open Forecasting Center (MOFC) at the University of Nicosia conducts cutting-edge forecasting research and provides business forecast training. It helps companies achieve accurate predictions, estimate the levels of uncertainty, avoiding costly mistakes, and apply best forecasting practices. The MOFC is well known for its Makridakis Competitions, the first of which ran in the 1980s.

In this competition, the fifth iteration, you will use hierarchical sales data from Walmart, the world’s largest company by revenue, to forecast daily sales for the next 28 days. The data, covers stores in three US States (California, Texas, and Wisconsin) and includes item level, department, product categories, and store details. In addition, it has explanatory variables such as price, promotions, day of the week, and special events. Together, this robust dataset can be used to improve forecasting accuracy.

If successful, your work will continue to advance the theory and practice of forecasting. The methods used can be applied in various business areas, such as setting up appropriate inventory or service levels. Through its business support and training, the MOFC will help distribute the tools and knowledge so others can achieve more accurate and better calibrated forecasts, reduce waste and be able to appreciate uncertainty and its risk implications.

Evaluation:
This competition uses a Weighted Root Mean Squared Scaled Error (RMSSE). Extensive details about the metric, scaling, and weighting can be found in the [M5 Participants Guide](https://mofc.unic.ac.cy/m5-competition/).

It is important to notice that this notebook must be seen together with the notebooks "M5 Forecasting - Accuracy Data study.ipynb" and "M5 Forecasting - Accuracy  - Variables". All the necessary data verification, the calculation of the parameters of the ARIMA(p, i, q), and the construction of the variables are done in those two notebook. This notebook have the correlations between variables, the ARIMA model with exogenous variables, and the forecast.

In [2]:
import gc
import time
import pandas as pd
import numpy as np
import re
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.stattools import acf
from statsmodels.tsa.stattools import pacf
from statsmodels.graphics.tsaplots import plot_acf
from statsmodels.graphics.tsaplots import plot_pacf

from statsmodels.tsa.arima_model import ARIMA

In [4]:
sales_train_validation = pd.read_csv('sales_train_validation.csv') #Only used to take the products ids.
products_ids = sales_train_validation['id'].unique()
products_ids_size = len(products_ids)
del sales_train_validation

First, we will calculate the correlation between the categories of the variables and the demand. We will keep only the categories qith a correlation higher than 0.05. After, we will calculate the correlation between variable and for each pair with a correlation higher than 0.5, we will keep only the categorie with higher correlation with the demand.
We could have done this step with the variables without being categorized, but this would imply that we would need to categorize every variable everytime we open a file to model (we would need to categorize the same same variable more than 30.000 times!). herefore, we decided to use the variables already categorized in this step. Nonetheles, the weakness of this strategy is that we may have some categories of one variable and some others categories from another variable.Considering that we will drop categories with a correlation higher than 0.5, this will not imply in any statistical error.

In [9]:
#Used ti filter variables by correlation
def get_redundant_pairs(df):
        '''Get diagonal and lower triangular pairs of correlation matrix'''
        pairs_to_drop = set()
        cols = df.columns
        for i in range(0, df.shape[1]):
            for j in range(0, i+1):
                pairs_to_drop.add((cols[i], cols[j]))
        return pairs_to_drop

In [198]:
'''Verify the correlation between variables:

Time variables:
    season_1, season_2, season_4
    month_fase_1, month_fase_3
    week_fase_1, week_fase_3
    month_1, month_2, month_3, month_4, month_6, month_7, month_8, month_9, month_10, month_11, month_12
    wday_1, wday_2, wday_3, wday_4, wday_6, wday_7

Holiday variables:
It can be identified in the database looking for columns that end with "_near", "_week", or "_weekend" 
Plus the variables:
    'SuperBowl', 'ValentinesDay', 'PresidentsDay', 'LentStart', 'LentWeek2',
    'StPatricksDay', 'Purim End', 'OrthodoxEaster', 'Pesach End',
    'Cinco De Mayo', 'Mother's day', 'MemorialDay', 'NBAFinalsStart',
    'NBAFinalsEnd', 'Father's day', 'IndependenceDay', 'Ramadan starts',
    'Eid al-Fitr', 'LaborDay', 'ColumbusDay', 'Halloween', 'EidAlAdha',
    'VeteransDay', 'Thanksgiving', 'Christmas', 'Chanukah End', 'NewYear',
    'OrthodoxChristmas', 'MartinLutherKingDay', 'Easter', 'Sporting',
    'Cultural', 'National', 'Religious'

Demand variables:
    demand_rolling_std_t7, demand_rolling_std_t30, demand_rolling_std_t90,
    demand_rolling_std_t180, demand_rolling_std_t365, demand_rolling_skew_t30, demand_rolling_kurt_t30

Price variables:
It can be identified in the database looking for columns tha with "price" in the name. 
The first column name is dropes because we will no use the "sell_price" variable.

Snap variables:
    snap, snap_other, snap_only_other
'''

def variable_correlation(df, variables, n):
    
    variables.append('demand')
    df = df[variables]
    corr_matrix = df.corr()
    sorted_corrs = corr_matrix['demand'].abs().sort_values(ascending = False)
    #delete variables with correlation lower than 0.05
    variables_to_keep = sorted_corrs[sorted_corrs > 0.05].index
    
 
    corr_table =  df[variables_to_keep].corr().abs().unstack()
    labels_to_drop = get_redundant_pairs(df[variables_to_keep])
    corr_table = corr_table.drop(labels = labels_to_drop).sort_values(ascending = False)
    variables_to_drop = []
    for e in corr_table[corr_table > 0.5].index:
        variables_to_drop.append(sorted_corrs[sorted_corrs == min(sorted_corrs[e[0]],sorted_corrs[e[1]])].index[0])
    
    variables_to_keep = variables_to_keep.drop(variables_to_drop)

    return variables_to_keep[1:]

In [229]:
'''Main code
First: variables selection analyzing the correlation.
Second: ARIMA(p, i, q) model
Third: Forecast
Last: holdout construction
'''
progress = 0   #Usefull to see the progress of the code
progress_1000 = 1
start = time.time()

path = 'C:\\Users\\maxwi\\Python\\Kaggle\\M5 Forecasting - Accuracy\\Modelo 1\\model\\data_by_id\\'
for e in products_ids[0:1]:
    data_variables = pd.read_hdf(path + e + '.h5')
    display(data_variables.head())
    print()
    columns_to_keep = ['demand']
    
    #Select variables:
    #Time variables:
    time_variables = ['season_1', 'season_2', 'season_4'
                 , 'month_fase_1', 'month_fase_3'
                 , 'week_fase_1', 'week_fase_3'
                 , 'month_1', 'month_2', 'month_3', 'month_4', 'month_6', 'month_7'
                 , 'month_8', 'month_9', 'month_10', 'month_11', 'month_12'
                 , 'wday_1', 'wday_2', 'wday_3', 'wday_4', 'wday_6', 'wday_7']
    variables_to_keep_time = variable_correlation(data_variables[data_variables['part'] == 'train'], time_variables, 30)
    for e in variables_to_keep_time.values:
        columns_to_keep.append(e)
    
    #Holiday variables:
    holidays_variables = ["SuperBowl", 'ValentinesDay', 'PresidentsDay', 'LentStart', 'LentWeek2',
       'StPatricksDay', 'Purim End', 'OrthodoxEaster', 'Pesach End',
       'Cinco De Mayo', "Mother's day", 'MemorialDay', 'NBAFinalsStart',
       'NBAFinalsEnd', "Father's day", 'IndependenceDay', 'Ramadan starts',
       'Eid al-Fitr', 'LaborDay', 'ColumbusDay', 'Halloween', 'EidAlAdha',
       'VeteransDay', 'Thanksgiving', 'Christmas', 'Chanukah End', 'NewYear',
       'OrthodoxChristmas', 'MartinLutherKingDay', 'Easter', 'Sporting',
       'Cultural', 'National', 'Religious']
    holidays_variables_2 = [col for col in data_variables.columns if col.endswith("_near") or col.endswith("_week") or col.endswith("_weekend")]
    for e in holidays_variables_2:
        holidays_variables.append(e)
    variables_to_keep_holidays = variable_correlation(data_variables[data_variables['part'] == 'train'], holidays_variables, 30)
    for e in variables_to_keep_holidays.values:
        columns_to_keep.append(e)
    
    #Demand variables:
    demand_variables = ['demand_rolling_std_t7', 'demand_rolling_std_t30', 'demand_rolling_std_t90'
                        , 'demand_rolling_std_t180', 'demand_rolling_std_t365', 'demand_rolling_skew_t30'
                        , 'demand_rolling_kurt_t30']
    variables_to_keep_demand = variable_correlation(data_variables[data_variables['part'] == 'train'], demand_variables, 30)
    for e in variables_to_keep_demand.values:
        columns_to_keep.append(e)
    
    #Price variables:
    price_variables = [col for col in data_variables.columns if 'price' in col][1:]
    variables_to_keep_price = variable_correlation(data_variables[data_variables['part'] == 'train'], price_variables, 30)
    for e in variables_to_keep_price.values:
        columns_to_keep.append(e)

        
    #Snap variables:
    snap_variables = ['snap', 'snap_other', 'snap_only_other']
    variables_to_keep_snap = variable_correlation(data_variables[data_variables['part'] == 'train'], snap_variables, 30)
    for e in variables_to_keep_snap.values:
        columns_to_keep.append(e)
    
    #Parameter for the ARIMA(p, i, q) model
    columns_to_keep.append('p_parameter')
    columns_to_keep.append('i_parameter')
    columns_to_keep.append('q_parameter')
    
    columns_to_keep.append('part') #Differenciate between train and test
    
    #ARIMA(p, i, q) model
    
    LEMBRAR DE SELECIONAR SOMENTE part = 'train'
    
    print()
    end = time.time()
    elapsed = end - start
    print(elapsed)

,day,demand,demand_rolling_kurt_t30,demand_rolling_skew_t30,demand_rolling_std_t180,demand_rolling_std_t30,demand_rolling_std_t365,demand_rolling_std_t7,demand_rolling_std_t90,id,...,wday_4,wday_6,wday_7,month_fase_1,month_fase_3,week_fase_1,week_fase_3,p_parameter,i_parameter,q_parameter
16838178,d_897,0,NaN,NaN,0.0,0.0,0.0,0.0,0.0,HOBBIES_1_001_CA_1_validation,...,0,0,0,0,0,1,0,0,0,0
16862809,d_898,0,NaN,NaN,0.0,0.0,0.0,0.0,0.0,HOBBIES_1_001_CA_1_validation,...,0,0,0,0,0,1,0,0,0,0
16887440,d_899,0,NaN,NaN,0.0,0.0,0.0,0.0,0.0,HOBBIES_1_001_CA_1_validation,...,0,0,0,0,0,0,0,0,0,0
16912071,d_900,0,NaN,NaN,0.0,0.0,0.0,0.0,0.0,HOBBIES_1_001_CA_1_validation,...,1,0,0,0,0,0,0,0,0,0
16936702,d_901,0,NaN,NaN,0.0,0.0,0.0,0.0,0.0,HOBBIES_1_001_CA_1_validation,...,0,0,0,0,0,0,0,0,0,0



['demand', 'wday_1', 'season_4', 'wday_6', 'season_2', 'LaborDay', 'PresidentsDay_weekend', 'ValentinesDay_weekend', 'Cultural_weekend', 'Easter_weekend', "Father's day_near", 'VeteransDay', 'National_weekend', 'Sporting_week', 'MemorialDay_week', 'Purim End_week', 'Ramadan starts_near', 'OrthodoxChristmas_week', 'MartinLutherKingDay_near', 'VeteransDay_near', 'NewYear_weekend', 'Christmas_weekend', "Mother's day_week", 'IndependenceDay_weekend', 'demand_rolling_std_t30', 'p_parameter', 'i_parameter', 'q_parameter', 'part']

0.12860941886901855


In [ ]:
#TEST ARIMA - Just an old scratch!
product = 'HOBBIES_1_003_CA_1_validation'
 
p = df_piq_table[df_piq_table['product_id'] == product]['p_parameter'].iloc[0]
i = df_piq_table[df_piq_table['product_id'] == product]['i_parameter'].iloc[0]
q = df_piq_table[df_piq_table['product_id'] == product]['q_parameter'].iloc[0]
print(p, i, q)
print()

demand_original = data[data['id'] == product]['demand'].values.tolist() #Used to fix the first difference in the forecast.
demand = data[data['id'] == product]['demand']
if i == 1:
    demand = demand.diff(periods = 1)[1:]
        

# fit model
model = ARIMA(demand, order = (p, i, q))

error_fit = []
try:
    model_fit = model.fit()
except:
    model = ARIMA(demand, order = (p, i, q))
    params = np.zeros(1 + p + q)
    model_fit = model.fit(start_params = params)
    error_fit.append(product)

    
#TEST Forecast
print('Forecast')
forecast = model_fit.forecast(steps = 28)[0]
if i == 0:
    forecast_2 = [round(e, 0) for e in forecast]
else:
    forecast_2 = [demand_original[-1]]
    for e in forecast:
        forecast_2.append(e + forecast_2[-1])
    forecast_2 = forecast_2[1:]
    for i in range(len(forecast_2)):
        if forecast_2[i] < 0:
            forecast_2[i] = 0
        else:
            forecast_2[i] = round(forecast_2[i], 0)
 
forecast_2